<a href="https://colab.research.google.com/github/ratmcu/wiki_ner/blob/master/info_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyahocorasick
!pip install fuzzyset
from ahocorasick import Automaton
import fuzzyset
import urllib
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
import re
import os
import pandas as pd
nltk.download('punkt')

     |████████████████████████████████| 317kB 5.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 378kB 4.9MB/s 
     |████████████████████████████████| 51kB 23.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/d8/36/9a/8f1cac047c7c3b03dce3d5434ed0088bfd8da8aeca615dfb4c
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Stored in directory: /root/.cache/pip/wheels/a7/4e/77/da46da4c5aece69b701cf45e6e4fe56ff98bfbdcc7b271d03b
Successfully built fuzzyset python-levenshtein texttable
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
class PageContents():
    def __init__(self, url):
        self.quote_page = url
        try:
            self.page = urllib.request.urlopen(self.quote_page)
            self.soup = BeautifulSoup(self.page, 'html.parser')
        except:
            self.table = False
      
        try:
            self.side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
            self.table_entry_list = self.side_pane.find('tbody').find_all('tr')
            self.table = True
        except:
            self.table = False
    #       print(self.table_entry_list)
    #       self.table_entry_list = self.soup.find('tbody').find_all('tr')
   
    def get_party(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
        party = ''
        for table_entry in self.table_entry_list:
            for child in table_entry.children:
                if child.text == 'Political party':
                    party = child.next_sibling.contents[0].get('title')
#                   print(party)

In [0]:
class InfoCard():
    def __init__(self, page_content):
        self.info_table = {}
        if not page_content.table:
            raise RuntimeError('page content is incomplete') from error
        for table_entry in page_content.table_entry_list:
            try:
                row_left = table_entry.find('th', attrs={'scope': 'row'})
                row_right = row_left.next_sibling
                self.info_table[row_left.text] = self.get_text_parts(self.get_kids(row_right))
            except:
                pass

        self.info_table = self.filter_info_scrapes(self.info_table)
        print('info card is scraped successfully')
        
    def get_kids(self, html_mother):
        kid_list = []
        try:
            kids = html_mother.children
            for kid in kids:
                kid_list.append(self.get_kids(kid))
        except:
            return html_mother
        return kid_list 

    def get_text_parts(self, text_lists):
        text_parts = []
        if type(text_lists) == list:
            for element in text_lists:
                text_parts.extend(self.get_text_parts(element))
        else:
            text_parts.append(text_lists)
        return(text_parts)

    def filter_info_scrapes(self, scape_dict):
        mask_dict = {}
        for key, val in scape_dict.items():
            mask_dict[key] = []
            for i, element in enumerate(scape_dict[key]):
                fnd = [c in element for c in u'()[]\n\xa0']
                if not True in fnd:
                    mask_dict[key].append(element)  
        return mask_dict    

In [5]:
# pg = PageContents('https://en.wikipedia.org/wiki/barack_Obama')
pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
info_card = InfoCard(pg)
info_card.info_table

info card is scraped successfully


{'Awards': ['List of honors and awards'],
 'Born': ['Donald John Trump',
  '1946-06-14',
  'June 14, 1946',
  'Queens',
  ', ',
  'New York City'],
 'Children': ['Donald Jr.', 'Ivanka', 'Eric', 'Tiffany', 'Barron'],
 'Education': ['The Wharton School', 'BS', ' in ', 'Econ.'],
 'Net worth': [],
 'Occupation': ['Politician',
  'businessman',
  'real estate developer',
  'television personality'],
 'Other politicalaffiliations': ['Democratic', 'Reform', 'Independent'],
 'Parents': ['Fred Trump', 'Mary Anne MacLeod'],
 'Political party': ['Republican'],
 'Preceded by': ['Barack Obama'],
 'Relatives': ['Trump family'],
 'Residence': ['White House', 'Trump Tower', 'Full list'],
 'Signature': [],
 'Spouse(s)': ['Ivana Zelníčková',
  'm.',
  '1977',
  '; ',
  'div.',
  '1992',
  'Marla Maples',
  'm.',
  '1993',
  '; ',
  'div.',
  '1999',
  'Melania Knauss',
  'm.',
  '2005'],
 'Vice President': ['Mike Pence'],
 'Website': ['Official website',
  'White House website',
  'Presidential Twitter'

In [10]:
def get_kids(html_mother):
    kid_list = []
    try:
        kids = html_mother.children
        for kid in kids:
            kid_list.append(get_kids(kid))
    except:
        return html_mother
    return kid_list

def get_text_parts(text_lists):
    text_parts = []
    if type(text_lists) == list:
        for element in text_lists:
            text_parts.extend(get_text_parts(element))
    else:
        text_parts.append(text_lists)
    return(text_parts)

def filter_info_scrapes(scape_dict):
    mask_dict = {}
    for key, val in scape_dict.items():
#         print(key)
#         print(scape_dict[key])
        mask_dict[key] = []
        for i, element in enumerate(scape_dict[key]):
#             print(element)
#             mask_dict[key].append(element)
            fnd = [c in element for c in u'()[]\n\xa0']
            if not True in fnd:
#                 print('!!!')
# #                 mask_dict[key][-1] = '1'
# #                 del val[i]
#             else:
                mask_dict[key].append(element)  
    return mask_dict

True

In [0]:
pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# pg = PageContents('https://en.wikipedia.org/wiki/Ashraf_Ghani')
# pg = PageContents('https://en.wikipedia.org/wiki/Oracle_Corporation')


In [25]:
info_table = {}
start = True

for table_entry in pg.table_entry_list:
#     for child in table_entry.children:
    if table_entry.text == 'Personal details':
        start = True
    if start:    
#             print(child.text)
        try:
            row_left = table_entry.find('th', attrs={'scope': 'row'})
            row_right = row_left.next_sibling
#             print(row_left.text)
#             print(row_right)
#             print(get_text_parts(get_kids(row_right)))
            info_table[row_left.text] = get_text_parts(get_kids(row_right))
#             break
        except:
            pass

# info_table
# mthr
info_table = filter_info_scrapes(info_table)
# print(info_table)
# info_table['Children'][0]
info_table


{'Awards': ['List of honors and awards'],
 'Born': ['Donald John Trump',
  '1946-06-14',
  'June 14, 1946',
  'Queens',
  ', ',
  'New York City'],
 'Children': ['Donald Jr.', 'Ivanka', 'Eric', 'Tiffany', 'Barron'],
 'Education': ['The Wharton School', 'BS', ' in ', 'Econ.'],
 'Net worth': [],
 'Occupation': ['Politician',
  'businessman',
  'real estate developer',
  'television personality'],
 'Other politicalaffiliations': ['Democratic', 'Reform', 'Independent'],
 'Parents': ['Fred Trump', 'Mary Anne MacLeod'],
 'Political party': ['Republican'],
 'Preceded by': ['Barack Obama'],
 'Relatives': ['Trump family'],
 'Residence': ['White House', 'Trump Tower', 'Full list'],
 'Signature': [],
 'Spouse(s)': ['Ivana Zelníčková',
  'm.',
  '1977',
  '; ',
  'div.',
  '1992',
  'Marla Maples',
  'm.',
  '1993',
  '; ',
  'div.',
  '1999',
  'Melania Knauss',
  'm.',
  '2005'],
 'Vice President': ['Mike Pence'],
 'Website': ['Official website',
  'White House website',
  'Presidential Twitter'

In [13]:
filter_info_scrapes(info_table)
info_table

{'Awards': ['List of honors and awards'],
 'Born': ['Donald John Trump',
  ' (',
  '1946-06-14',
  ') ',
  'June 14, 1946',
  ' (age\xa072)',
  'Queens',
  ', ',
  'New York City'],
 'Children': ['\n',
  'Donald Jr.',
  '\n',
  'Ivanka',
  '\n',
  'Eric',
  '\n',
  'Tiffany',
  '\n',
  'Barron',
  '\n'],
 'Education': ['The Wharton School', ' (', 'BS', ' in ', 'Econ.', ')'],
 'Net worth': ['US$3.1\xa0billion (March 2019)', '[a]'],
 'Occupation': ['Politician',
  'businessman',
  'real estate developer',
  'television personality'],
 'Other politicalaffiliations': ['\n',
  'Democratic',
  ' (until 1987, 2001–2009)',
  '\n',
  'Reform',
  ' (1999–2001)',
  '\n',
  'Independent',
  ' (2011–2012)',
  '\n'],
 'Parents': ['Fred Trump', 'Mary Anne MacLeod'],
 'Political party': ['Republican', ' (1987–1999, 2009–2011, 2012–present)'],
 'Preceded by': ['Barack Obama'],
 'Relatives': ['Trump family'],
 'Residence': ['\n',
  'White House',
  ' (official)',
  '\n',
  'Trump Tower',
  ' (personal)'

In [14]:
[c in info_table['Spouse(s)'][-1] for c in u'()[],;.\n']

[False, False, False, False, False, False, False, True]